In [110]:
import pandas as pd
from google.cloud import bigquery
from datetime import datetime, timedelta
import joblib
import numpy as np

In [111]:
# project_id = "banded-setting-428309-q4"
# dataset_id = "datos"

In [112]:
# client = bigquery.Client(project='banded-setting-428309-q4')
# one_week_ago = datetime.now() - timedelta(days=7)
    
# query = f"""
# SELECT
#     FORMAT_TIMESTAMP('%Y-%m-%d', Timestamp) AS Day,
#     FORMAT_TIMESTAMP('%H', Timestamp) AS Hour,
#     FORMAT_TIMESTAMP('%M', Timestamp) AS Minute,
#     ct.descripcion,
#     bd.Value
# FROM `banded-setting-428309-q4.datos.bronze-data` bd
# LEFT JOIN `banded-setting-428309-q4.datos.col-tag` ct on bd.Tag = ct.tag
# WHERE DATE(Timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 50 DAY) AND CURRENT_DATE()
# """

# query_job = client.query(query)
# results = query_job.result()

# data = []   
# for row in results:     
#     data.append(dict(row))
    
# df = pd.DataFrame(data)

In [113]:
# df.head()

In [114]:
# df.to_parquet('140_days_df.parquet')

In [115]:
df = pd.read_parquet('140_days_df.parquet')

In [116]:
df_max_values = df.groupby(["descripcion", "Day", "Hour", "Minute"]).agg({"Value": "max"}).reset_index()

df_max_values['dayhourminute'] = df_max_values['Day'] + ' ' + df_max_values['Hour'] + ':' + df_max_values['Minute']
df_unpivot = df_max_values.pivot_table(index="dayhourminute", columns="descripcion", values="Value", aggfunc="max").reset_index()

In [117]:
df_unpivot

descripcion,dayhourminute,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,...,VOL RECUPERACION ANION GRUPO H0,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A EDAR,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,W10 STATUS,W7 STATUS
0,2024-06-09 00:00,0.000000,NaN,852.856995,9697.578125,NaN,NaN,51.768593,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-09 00:01,11.125526,NaN,1051.594116,10000.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-06-09 00:02,24.037436,NaN,1047.765991,9903.143555,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-09 00:03,27.494791,NaN,979.953308,10000.000000,60.657055,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-06-09 00:04,46.029263,NaN,899.699402,10000.000000,NaN,54.669937,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18212,2024-06-21 15:48,0.000000,NaN,NaN,NaN,NaN,14.933886,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18213,2024-06-21 15:49,NaN,NaN,NaN,0.000000,NaN,16.333483,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18214,2024-06-21 15:50,11.963161,NaN,NaN,4456.027344,NaN,15.153301,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18215,2024-06-21 15:51,20.796946,NaN,NaN,4819.892578,NaN,18.225300,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
columns_to_check = [
    "COR TITÁNIC AZÚCARES", 
    "COT TITÁNIC AZÚCARES NUEVO", 
    "COT AGUAS ÁCIDAS", 
    "COT AGUAS ÁCIDAS NUEVO"
]

conditions = pd.Series([False] * len(df_unpivot))
for col in columns_to_check:
    if col in df_unpivot.columns:
        conditions |= (df_unpivot[col] > 2500)

df_unpivot["flag"] = conditions.astype(int)

df_unpivot["dayhourminute"] = pd.to_datetime(df_unpivot["dayhourminute"])

df_unpivot.head()


descripcion,dayhourminute,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,...,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A EDAR,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,W10 STATUS,W7 STATUS,flag
0,2024-06-09 00:00:00,0.000000,NaN,852.856995,9697.578125,NaN,NaN,51.768593,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2024-06-09 00:01:00,11.125526,NaN,1051.594116,10000.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2024-06-09 00:02:00,24.037436,NaN,1047.765991,9903.143555,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2024-06-09 00:03:00,27.494791,NaN,979.953308,10000.000000,60.657055,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2024-06-09 00:04:00,46.029263,NaN,899.699402,10000.000000,NaN,54.669937,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [119]:
col_drop = ['COT AGUAS ÁCIDAS NUEVO', 'COT AGUAS ÁCIDAS', 'COR TITÁNIC AZÚCARES', 'COT TITÁNIC AZÚCARES NUEVO','dayhourminute']  
df = df_unpivot.drop(columns=[col for col in col_drop if col in df_unpivot.columns])
df = df.fillna(0)

In [120]:
exclude_columns = ['flag']
df_corr = df.drop(columns=exclude_columns).corr().abs()

upper_triangle_mask = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))

drop_columns = set()

for column in upper_triangle_mask.columns:
    high_corr = upper_triangle_mask[column][(upper_triangle_mask[column] > 0.8) & (upper_triangle_mask[column] < 1)]
    if not high_corr.empty:
        drop_columns.add(column)

df = df.drop(columns=list(drop_columns))

In [121]:
# one_hour_earlier = df_unpivot["dayhourminute"] - pd.Timedelta(hours=1)
# df_unpivot["flag"] |= df_unpivot["dayhourminute"].isin(one_hour_earlier[conditions])

# df_unpivot["flag"] = df_unpivot["flag"].astype(int)

# df_unpivot.head()

In [122]:
df

descripcion,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,BOMBA RECUPERACIÓN GRUPOS H0,...,VOL RECUP ANION GRUPOS GLUC,VOL RECUP CATION GRUPOS GLUC,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,W7 STATUS,flag
0,0.000000,0.0,852.856995,9697.578125,0.000000,0.000000,51.768593,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,11.125526,0.0,1051.594116,10000.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,24.037436,0.0,1047.765991,9903.143555,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,27.494791,0.0,979.953308,10000.000000,60.657055,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,46.029263,0.0,899.699402,10000.000000,0.000000,54.669937,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18212,0.000000,0.0,0.000000,0.000000,0.000000,14.933886,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
18213,0.000000,0.0,0.000000,0.000000,0.000000,16.333483,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
18214,11.963161,0.0,0.000000,4456.027344,0.000000,15.153301,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
18215,20.796946,0.0,0.000000,4819.892578,0.000000,18.225300,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [123]:
X = df.drop('flag', axis=1)
X = X.fillna(0)
X.head()

descripcion,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,BOMBA RECUPERACIÓN GRUPOS H0,...,VLV REGUL NIVEL CUBA MEZCLA DC,VOL RECUP ANION GRUPOS GLUC,VOL RECUP CATION GRUPOS GLUC,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,W7 STATUS
0,0.000000,0.0,852.856995,9697.578125,0.000000,0.000000,51.768593,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.125526,0.0,1051.594116,10000.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24.037436,0.0,1047.765991,9903.143555,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27.494791,0.0,979.953308,10000.000000,60.657055,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,46.029263,0.0,899.699402,10000.000000,0.000000,54.669937,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
y = df['flag']
y.value_counts()

flag
0    15464
1     2753
Name: count, dtype: int64

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [126]:
model = LogisticRegression()

# Entrenar el modelo
model.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred = model.predict(X_test_scaled)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(cm)
print('Classification Report:')
print(report)

Accuracy: 0.8504390779363337
Confusion Matrix:
[[3048   54]
 [ 491   51]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      3102
           1       0.49      0.09      0.16       542

    accuracy                           0.85      3644
   macro avg       0.67      0.54      0.54      3644
weighted avg       0.81      0.85      0.80      3644



In [127]:
import joblib
model_filename = 'logistic_model.pkl'
joblib.dump(model, model_filename)
print(f'Model saved as {model_filename}')

Model saved as logistic_model.pkl


In [128]:
sclaer_filename = 'scaler_model.pkl'
joblib.dump(scaler, sclaer_filename)
print(f'Scaler saved as {sclaer_filename}')

Scaler saved as scaler_model.pkl


# CHECK

## COEFFICIENTS

In [129]:
coefficients = model.coef_[0]
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'Feature {i}' for i in range(X_train_scaled.shape[1])]

top_20_indices = np.argsort(np.abs(coefficients))[-20:]

print("Top 20 coefficients (absolute terms):")
for idx in reversed(top_20_indices):
    print(f'Feature {feature_names[idx]}: {coefficients[idx]}')


Top 20 coefficients (absolute terms):
Feature FASES REGEN CAT-ANI G.R 7: -0.33322028736833054
Feature PRESION ENTRADA FILTRO GAF W7: 0.30355332233844756
Feature ESTADO CIRCUITO SECADERO DX: -0.23124083054967917
Feature NIVEL 1 TORTA P45 H0: -0.2245912049466538
Feature NIVEL CUBA G-2: -0.21788484372537714
Feature PRESION DIFERENCIAL FILT GAF W7: -0.21312799144752181
Feature NIVEL CUBA SOSA DILUIDA: 0.19532397714891464
Feature PRESION FILTRO NIAGRA Nº4 H0: -0.19205442173178458
Feature NIVEL CUBA C5: -0.18479021684902963
Feature PRESION DIFERENCIAL FILT GAF W10: 0.18460390510165667
Feature % BOMBA ALIMENTACION CUBA M7: -0.1796400618092394
Feature PRESIÓN DIF PRENSA ALIM.35/35/09: 0.17924156712062847
Feature NIVEL CUBA TIERRAS NIAGARA H0: -0.1766112587971757
Feature FASES REGEN CATION G.R 5: -0.176162488018148
Feature NIVEL TAMIZ TITANIC AH (PV): 0.16805989537485394
Feature NIVEL CUBA BASE "B": -0.16682925308359917
Feature POT BOMBA VACIO Nº1 P45 GL: -0.1619345853570831
Feature FASES REGEN

## CORRECT OBSERVATIONS

In [130]:
X_scaled = scaler.transform(X)

df['predicted'] = model.predict(X_scaled) 

df['flag'] = df['predicted'] == df[y_test.name]
correct_predictions = df['flag'].sum()

print(f'Number of correctly predicted rows: {correct_predictions}')

total_predictions = len(df)
percentage_correct = (correct_predictions / total_predictions) * 100

print(f'Percentage of correctly predicted rows: {percentage_correct:.2f}%')

Number of correctly predicted rows: 15529
Percentage of correctly predicted rows: 85.24%


In [133]:
correct_predictions_0 = df[df['predicted'] == 0]['flag'].sum()
total_predictions_0 = df[df['predicted'] == 0].shape[0]
percentage_correct_0 = (correct_predictions_0 / total_predictions_0) * 100 if total_predictions_0 > 0 else 0

correct_predictions_1 = df[df['predicted'] == 1]['flag'].sum()
total_predictions_1 = df[df['predicted'] == 1].shape[0]
percentage_correct_1 = (correct_predictions_1 / total_predictions_1) * 100 if total_predictions_1 > 0 else 0

print(f'Percentage of correctly predicted 0s: {percentage_correct_0:.2f}%')
print(f'Percentage of correctly predicted 1s: {percentage_correct_1:.2f}%')

Percentage of correctly predicted 0s: 86.20%
Percentage of correctly predicted 1s: 55.67%


In [131]:
df

descripcion,% BOMBA ALIM P45 M2 GLUCOSA,% BOMBA ALIMENT COLUMNAS 353509,% BOMBA ALIMENTACION CUBA M7,% BOMBA ALIMENTACION P45 M2 H0,% REG. VF BOMBA ALIMENT. SWENSON,% VAR. BOMBA SDA. CUBETA CH,% VARIADOR BOMBA ALIM. C.V.,% VF BOMBA ALIMENTACION W7,% VF BOMBA SALIDA SWENSON A CCHH,BOMBA RECUPERACIÓN GRUPOS H0,...,VOL RECUP CATION GRUPOS GLUC,VOL. RECUP. ANIÓN GRUPO 4,VOL. RECUP. CATIÓN GRUPO 4,VOLUMEN CIP A PEQUEÑAS AGUAS,VOLUMEN CIP LAVADO AGUA,VOLUMEN LAVADO CIP,VÁLVULA REGULADORA NIVEL M10,W7 STATUS,flag,predicted
0,0.000000,0.0,852.856995,9697.578125,0.000000,0.000000,51.768593,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0
1,11.125526,0.0,1051.594116,10000.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0
2,24.037436,0.0,1047.765991,9903.143555,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0
3,27.494791,0.0,979.953308,10000.000000,60.657055,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0
4,46.029263,0.0,899.699402,10000.000000,0.000000,54.669937,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18212,0.000000,0.0,0.000000,0.000000,0.000000,14.933886,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0
18213,0.000000,0.0,0.000000,0.000000,0.000000,16.333483,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0
18214,11.963161,0.0,0.000000,4456.027344,0.000000,15.153301,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0
18215,20.796946,0.0,0.000000,4819.892578,0.000000,18.225300,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0
